In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

https://pytorch.org/tutorials/beginner/chatbot_tutorial.html?highlight=chatbot#define-evaluation

# Import

In [2]:
from pathlib import Path
import io

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torchtext
from torchtext.vocab import build_vocab_from_iterator, GloVe
from nltk.translate import bleu

import torch
from torch import nn

from torch.utils.data import Dataset, DataLoader
from typing import List

/home/riyadh/codes/nlp/learn2ask/venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Look at Data
Data already preprocessed collected from the original repo.

In [3]:
data_root = Path("data/processed")

In [4]:
with open(data_root / "src-train.txt") as f:
    src_train = [line.strip() for line in f]

with open(data_root / "tgt-train.txt") as f:
    tgt_train = [line.strip() for line in f]

with open(data_root / "src-dev.txt") as f:
    src_dev = [line.strip() for line in f]

with open(data_root / "tgt-dev.txt") as f:
    tgt_dev = [line.strip() for line in f]

with open(data_root / "src-test.txt") as f:
    src_test = [line.strip() for line in f]

with open(data_root / "tgt-test.txt") as f:
    tgt_test = [line.strip() for line in f]

print(len(src_train), len(src_dev), len(src_test))

70484 10570 11877


# Vocabulary

In [5]:
import datahandler as dh

In [6]:
vocab = dh.load_and_build_vocab(
    data_root / "src-train.txt", data_root / "tgt-train.txt"
)

In [7]:
pad_index = vocab["<PAD>"]
sos_index = vocab["<SOS>"]
eos_index = vocab["<EOS>"]

In [8]:
# a tensor of shape (vocab_size, embedding_dim)
embedding_vector = dh.load_pretrained_glove(vocab, cache="data/")

# Batch Dataloader

In [9]:
config = {"src_max_seq": 150, "tgt_max_seq": 50, "batch_size": 64, "lr": 1.0}

In [10]:
from tokenization import Tokenizer

In [40]:
tokenizer = Tokenizer(vocab, pad_index, sos_index, eos_index)

src_train_tensor, src_train_mask = tokenizer.encode(
    src_train, max_seq=config["src_max_seq"]
)

tgt_train_tensor, tgt_train_mask = tokenizer.encode(
    tgt_train, add_sos=True, add_eos=True, max_seq=config["tgt_max_seq"]
)

src_test_tensor, src_test_mask = tokenizer.encode(
    src_test, max_seq=config["src_max_seq"]
)
tgt_test_tensor, tgt_test_mask = tokenizer.encode(
    tgt_test, add_sos=True, add_eos=True, max_seq=config["tgt_max_seq"]
)

src_dev_tensor, src_dev_mask = tokenizer.encode(src_dev, max_seq=config["src_max_seq"])
tgt_dev_tensor, tgt_dev_mask = tokenizer.encode(
    tgt_dev, add_sos=True, add_eos=True, max_seq=config["tgt_max_seq"]
)

In [12]:
tokenizer.decode(src_train_tensor[0].unsqueeze(dim=0), keep_specials=False)

['a pub / <UNK> / , or public house is , despite its name , a private house , but is called a public house because it is licensed to sell alcohol to the general public .']

In [13]:
src_train[0]

'a pub / pʌb / , or public house is , despite its name , a private house , but is called a public house because it is licensed to sell alcohol to the general public .'

In [14]:
src_train_mask.shape

torch.Size([70484, 150])

In [41]:
class SentenceQuestionDataset(Dataset):
    def __init__(
        self,
        sentences: torch.Tensor,
        questions: torch.Tensor,
        sentences_mask=None,
        questions_mask=None,
    ):
        """
        Represents a dataset of text pairs for training or evaluating models that
        deal with relationships between text passages.

        Args:
            vocab (torchtext.vocab.Vocab): A pre-built vocabulary object
                containing word mappings from text to numerical representation.
            sentences (List[str]): A list of text passages (sentences, paragraphs, etc.).
            questions (List[str]): A list of corresponding questions related to the sentences.
            Ls (int, optional): The maximum length to which sentences will be
                truncated or padded during preprocessing (default: 150).
            Lq (int, optional): The maximum length to which questions will be
                truncated or padded during preprocessing (default: 50).
        """
        self.sentences = sentences
        self.questions = questions
        self.sentences_mask = sentences_mask
        self.questions_mask = questions_mask

    def __len__(self):
        return self.sentences.size(0)

    def __getitem__(self, index):
        return (
            self.sentences[index],
            self.questions[index],
            self.sentences_mask[index],
            self.questions_mask[index],
        )

In [42]:
train_ds = SentenceQuestionDataset(
    src_train_tensor, tgt_train_tensor, src_train_mask, tgt_train_mask
)
test_ds = SentenceQuestionDataset(
    src_test_tensor, tgt_train_tensor, src_test_mask, tgt_test_mask
)
dev_ds = SentenceQuestionDataset(
    src_dev_tensor, tgt_dev_tensor, src_dev_mask, tgt_dev_mask
)

In [43]:
train_dl = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=True)
print(len(train_dl))
dev_dl = DataLoader(dev_ds, batch_size=8, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

1102


# Models

In [18]:
from models import Encoder, Decoder, Seq2SeqEncoderDecoder

# Trainer

In [19]:
from trainer import Trainer

In [53]:
config["lr"] = 1.0

net = Seq2SeqEncoderDecoder(
    vocab_size=len(vocab),
    embedding_vector=embedding_vector,
    embedding_dim=300,
    pad_index=pad_index,
    sos_index=sos_index,
    eos_index=eos_index,
    hidden_dim=8,
    bidirectional=True,
    num_layers=2,
)

optim = torch.optim.SGD(net.parameters(), lr=config["lr"])

# halve the learning rate once reaching epoch 8 as the original paper
lr_scheduler = torch.optim.lr_scheduler.MultiplicativeLR(
    optim, lr_lambda=lambda epoch: 0.5 if epoch > 8 else 1.0
)

trainer = Trainer(
    net,
    optim,
    train_dl,
    len(vocab),
    config["lr"],
    dev_dl,
    enable_checkpoint=True,
    limit_on_train_batch=2,
    enable_comet=True,
    comet_project="learn-to-ask-paper",
    comet_experiment_name=None,
    cfg=config,
    validation_data=tgt_dev,
    tokenizer=tokenizer,
    lr_scheduler=lr_scheduler,
)

trainer.fit(4)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/riyadhrazzaq/learn-to-ask-paper/ef3d47db24614a3c80ef23a6762a0535

  0%|▏                                                                                                                       | 2/1102 [00:14<2:10:51,  7.14s/batch, loss=10.8]


Epoch: 1,	Training loss: 10.779916763305664,	Validation metric: 45137.9140625	Time elapsed: 14.60 s


  0%|▏                                                                                                                       | 2/1102 [00:14<2:11:52,  7.19s/batch, loss=10.7]


Epoch: 2,	Training loss: 10.691727002461752,	Validation metric: 40273.1875	Time elapsed: 14.87 s


  0%|▏                                                                                                                       | 2/1102 [00:14<2:14:00,  7.31s/batch, loss=10.5]


Epoch: 3,	Training loss: 10.56266721089681,	Validation metric: 34029.01953125	Time elapsed: 15.04 s


  0%|▏                                                                                                                       | 2/1102 [00:15<2:20:21,  7.66s/batch, loss=10.4]
COMET INFO: ---------------------------------------------------------------------------------------                                                           
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/riyadhrazzaq/learn-to-ask-paper/ef3d47db24614a3c80ef23a6762a0535
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train/loss [4] : (10.436076164245605, 10.779916763305664)
COMET INFO:     val/loss [4]   : (10.36989974975586, 10.717477798461914)
COMET INFO:     val/pplx [4]   : (31885.28125, 45137.9140625)
COMET INFO:   Others:
COMET INFO:     Name : None
COMET INFO:   Parameters:
COMET INFO

Epoch: 4,	Training loss: 10.436076164245605,	Validation metric: 31885.28125	Time elapsed: 15.84 s


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET INFO: Uploading 1 metrics, params and output messages


In [ ]:
trainer.test(test_dl, tgt_test)

 27%|███████████████████████████▋                                                                          | 101/372 [02:35<07:16,  1.61s/it, bleu=0, loss=10.7, pplx=4.22e+4]

In [50]:
from time import sleep

In [52]:
for batch in test_dl:
    batch = [b.to(device) for b in batch]
    break

NameError: name 'device' is not defined